In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

%pylab inline
pylab.rcParams['figure.figsize'] = (20.0, 10.0)

%load_ext autoreload
%autoreload 2

sys.path.append('..')

import isolation
import sample_players
import run_match
import my_baseline_player as custom

Populating the interactive namespace from numpy and matplotlib


In [2]:
isolation.isolation._ACTIONSET

{<Action.SSE: -27>,
 <Action.SSW: -25>,
 <Action.ESE: -15>,
 <Action.WSW: -11>,
 <Action.ENE: 11>,
 <Action.WNW: 15>,
 <Action.NNE: 25>,
 <Action.NNW: 27>}

## A table could be made to have "state-action" pairs as key, and winning frequency as value

In [3]:
book = dict()

In [23]:
class ProbePlayer(custom.CustomPlayer):
    """ A class that is used to get statistics about the game."""
    def __init__(self, player_id):
        super().__init__(player_id)
        self.book_depth = 4
        self.context = dict()
        self.context['book'] = dict()
    
    def get_action(self, state):
        print('Player {} has played like this: {}'.format(self.player_id,
                                                           self.context['book']))
        depth = 1
        while True:
            action = self.alpha_beta_search(state, depth)
            if depth <= self.book_depth:
                self.context['book'][state.ply_count] = (state.board, state.locs, action)
            self.queue.put(action)
            depth += 1

In [24]:
state = isolation.isolation.Isolation()

In [25]:
state

Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None))

In [20]:
state.board

41523161203939122082683632224299007

In [26]:
# Custom vs Custom
from time import time

run_match.TIME_LIMIT = 150

num_rounds = 1
num_procs = 4
agent1 = isolation.Agent(ProbePlayer, "ProbePlayer1")
agent2 = isolation.Agent(ProbePlayer, "ProbePlayer2")

tic = time()
wins, num_games = run_match.play_matches(agent1, 
                                         agent2, 
                                         num_rounds, 
                                         num_procs, 
                                         fair_matches=False)
toc = time()
print('Wins: {}, Games: {}, Ratio: {}'.format(wins, 
                                              num_games, 
                                              wins / num_games))
print('Total time: {}, Time per game: {}'.format((toc-tic), (toc-tic)/num_games))

Running 2 games:
Player 0 has played like this: {}
Player 0 has played like this: {}
Player 1 has played like this: {}
Player 1 has played like this: {}
Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28)}
Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28)}
Player 1 has played like this: {1: (41523161203939122082683631955863551, (28, None), 29)}
Player 1 has played like this: {1: (41523161203939122082683631955863551, (28, None), 29)}
Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>)}
Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>)}
Player 1 has played like this: {1: (41523161203939122082683631955863551, (28, None), 29), 3: (41523161203939122073676432164251647, (53, 29), <Action.NNE: 25>)}

Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>), 4: (41523161203939122055662033654769663, (53, 54), <Action.NNE: 25>), 6: (41523161203032427690951061773740031, (78, 79), <Action.WSW: -11>), 8: (41523161203031984969093292744501247, (67, 68), <Action.SSW: -25>), 10: (41523161203031984969080098604967935, (42, 43), <Action.SSE: -27>), 12: (41523161203031984969080098604673023, (15, 18), <Action.NNE: 25>), 14: (41523161203031984969080098604672879, (4, 7), <Action.WNW: 15>)}
Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>), 4: (41523161203939122055662033654769663, (53, 54), <Action.NNE: 25>), 6: (41523161203032427690951061773740031, (78, 79), <Action.WSW: -11>), 8: (41523161203031984969093292744501247, (67, 68), <Action.SSW: -25>), 10: (41523161203031984969080098604967935

Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 16: (41523161203031984969080094309181295, (19, 32), <Action.NNE: 25>), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>), 4: (41523161203939122055662033654769663, (53, 54), <Action.NNE: 25>), 6: (41523161203032427690951061773740031, (78, 79), <Action.WSW: -11>), 8: (41523161203031984969093292744501247, (67, 68), <Action.SSW: -25>), 20: (41523161203031984824947304383604591, (33, 30), <Action.NNE: 25>), 10: (41523161203031984969080098604967935, (42, 43), <Action.SSE: -27>), 12: (41523161203031984969080098604673023, (15, 18), <Action.NNE: 25>), 18: (41523161203031984824947314047281007, (44, 57), <Action.WSW: -11>), 14: (41523161203031984969080098604672879, (4, 7), <Action.WNW: 15>)}
Player 1 has played like this: {19: (41523161203031984824947305457346415, (33, 57), <Action.SSE: -27>), 1: (41523161203939122082683631955863551, (28, None), 29), 3: (4152316120393912207367643216425164

Player 1 has played like this: {1: (41523161203939122082683631955863551, (28, None), 29), 3: (41523161203939122073676432164251647, (53, 29), <Action.NNE: 25>), 5: (41523161203636890600758376361093119, (78, 54), <Action.WSW: -11>), 7: (41523161203032280116998472097327103, (67, 79), <Action.WSW: -11>), 9: (41523161203031984969088894697990143, (42, 68), <Action.SSW: -25>), 11: (41523161203031984969080098604935167, (15, 43), <Action.SSW: -25>), 13: (41523161203031984969080098604673007, (4, 18), <Action.WNW: 15>), 15: (41523161203031984969080098604148591, (19, 7), <Action.NNE: 25>), 17: (41523161203031984969062502123136879, (44, 32), <Action.NNE: 25>), 19: (41523161203031984824947305457346415, (33, 57), <Action.SSE: -27>), 21: (41523161203031984536716928231892847, (58, 30), <Action.NNE: 25>), 23: (41523161193360577943771097815279471, (83, 55), <Action.WNW: 15>), 25: (41523161193359397280092786366048111, (56, 70), <Action.WSW: -11>)}
Player 0 has played like this: {0: (4152316120393912208268

Player 1 has played like this: {1: (41523161203939122082683631955863551, (28, None), 29), 3: (41523161203939122073676432164251647, (53, 29), <Action.NNE: 25>), 5: (41523161203636890600758376361093119, (78, 54), <Action.WSW: -11>), 7: (41523161203032280116998472097327103, (67, 79), <Action.WSW: -11>), 9: (41523161203031984969088894697990143, (42, 68), <Action.SSW: -25>), 11: (41523161203031984969080098604935167, (15, 43), <Action.SSW: -25>), 13: (41523161203031984969080098604673007, (4, 18), <Action.WNW: 15>), 15: (41523161203031984969080098604148591, (19, 7), <Action.NNE: 25>), 17: (41523161203031984969062502123136879, (44, 32), <Action.NNE: 25>), 19: (41523161203031984824947305457346415, (33, 57), <Action.SSE: -27>), 21: (41523161203031984536716928231892847, (58, 30), <Action.NNE: 25>), 23: (41523161193360577943771097815279471, (83, 55), <Action.WNW: 15>), 25: (41523161193359397280092786366048111, (56, 70), <Action.WSW: -11>), 27: (41523161193357035520390599240017775, (71, 59), <Actio

...

Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 64: (29026250130479421198052604560557835, (101, 110), <Action.ENE: 11>), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>), 4: (41523161203939122055662033654769663, (53, 54), <Action.NNE: 25>), 6: (41523161203032427690951061773740031, (78, 79), <Action.WSW: -11>), 8: (41523161203031984969093292744501247, (67, 68), <Action.SSW: -25>), 10: (41523161203031984969080098604967935, (42, 43), <Action.SSE: -27>), 12: (41523161203031984969080098604673023, (15, 18), <Action.NNE: 25>), 66: (23833913657863336437353311459362571, (112, 95), <Action.SSW: -25>), 14: (41523161203031984969080098604672879, (4, 7), <Action.WNW: 15>), 16: (41523161203031984969080094309181295, (19, 32), <Action.NNE: 25>), 18: (41523161203031984824947314047281007, (44, 57), <Action.WSW: -11>), 20: (41523161203031984824947304383604591, (33, 30), <Action.NNE: 25>), 22: (41523161203031984500688131212928879, (58, 55), <A

Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 64: (29026250130479421198052604560557835, (101, 110), <Action.ENE: 11>), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>), 4: (41523161203939122055662033654769663, (53, 54), <Action.NNE: 25>), 6: (41523161203032427690951061773740031, (78, 79), <Action.WSW: -11>), 8: (41523161203031984969093292744501247, (67, 68), <Action.SSW: -25>), 10: (41523161203031984969080098604967935, (42, 43), <Action.SSE: -27>), 12: (41523161203031984969080098604673023, (15, 18), <Action.NNE: 25>), 66: (23833913657863336437353311459362571, (112, 95), <Action.SSW: -25>), 14: (41523161203031984969080098604672879, (4, 7), <Action.WNW: 15>), 16: (41523161203031984969080094309181295, (19, 32), <Action.NNE: 25>), 18: (41523161203031984824947314047281007, (44, 57), <Action.WSW: -11>), 20: (41523161203031984824947304383604591, (33, 30), <Action.NNE: 25>), 22: (41523161203031984500688131212928879, (58, 55), <A

Player 0 has played like this: {0: (41523161203939122082683632224299007, (None, None), 28), 64: (29026250130479421198052604560557835, (101, 110), <Action.ENE: 11>), 2: (41523161203939122082683631418992639, (28, 29), <Action.NNE: 25>), 4: (41523161203939122055662033654769663, (53, 54), <Action.NNE: 25>), 6: (41523161203032427690951061773740031, (78, 79), <Action.WSW: -11>), 8: (41523161203031984969093292744501247, (67, 68), <Action.SSW: -25>), 10: (41523161203031984969080098604967935, (42, 43), <Action.SSE: -27>), 12: (41523161203031984969080098604673023, (15, 18), <Action.NNE: 25>), 66: (23833913657863336437353311459362571, (112, 95), <Action.SSW: -25>), 14: (41523161203031984969080098604672879, (4, 7), <Action.WNW: 15>), 16: (41523161203031984969080094309181295, (19, 32), <Action.NNE: 25>), 18: (41523161203031984824947314047281007, (44, 57), <Action.WSW: -11>), 20: (41523161203031984824947304383604591, (33, 30), <Action.NNE: 25>), 22: (41523161203031984500688131212928879, (58, 55), <A

Player 1 has played like this: {1: (41523161203939122082683631955863551, (28, None), 29), 3: (41523161203939122073676432164251647, (53, 29), <Action.NNE: 25>), 5: (41523161203636890600758376361093119, (78, 54), <Action.WSW: -11>), 51: (40711453788929854151349823835490059, (21, 45), <Action.NNW: 27>), 7: (41523161203032280116998472097327103, (67, 79), <Action.WSW: -11>), 9: (41523161203031984969088894697990143, (42, 68), <Action.SSW: -25>), 11: (41523161203031984969080098604935167, (15, 43), <Action.SSW: -25>), 13: (41523161203031984969080098604673007, (4, 18), <Action.WNW: 15>), 15: (41523161203031984969080098604148591, (19, 7), <Action.NNE: 25>), 17: (41523161203031984969062502123136879, (44, 32), <Action.NNE: 25>), 67: (40305644546153368029881780179846667, (91, 23), <Action.NNE: 25>), 21: (41523161203031984536716928231892847, (58, 30), <Action.NNE: 25>), 43: (40711453788929854151455379102451563, (17, 41), <Action.ENE: 11>), 23: (41523161193360577943771097815279471, (83, 55), <Action.

+
Wins: 1, Games: 2, Ratio: 0.5
Total time: 13.80253005027771, Time per game: 6.901265025138855


In [28]:
agent1[0].context

AttributeError: type object 'ProbePlayer' has no attribute 'context'

In [30]:
root = isolation.isolation.Isolation()

In [32]:
len(root.actions())

99

In [47]:
def get_full_tree(state, depth, s_a_set=None):
    if s_a_set == None:
        s_a_set = set()
    for action in state.actions():
        s_a_set.add((state, action))
        if depth > 1:
            s_a_set |= get_full_tree(state.result(action), depth - 1, s_a_set)
    return s_a_set

In [48]:
root = isolation.isolation.Isolation()
tree = get_full_tree(root, 1)

In [49]:
len(tree)

99

In [51]:
from time import time

times = list()
for depth in range(1,5):
    tic = time()
    root = isolation.isolation.Isolation()
    tree = get_full_tree(root, depth)
    toc = time()
    times.append(toc-tic)

In [52]:
times

[0.00012803077697753906,
 0.019460201263427734,
 0.2030651569366455,
 1.6986141204833984]

In [53]:
tree

{(Isolation(board=41198642638191437159754184456660991, ply_count=3, locs=(108, 83)),
  <Action.ESE: -15>),
 (Isolation(board=40143957350286345584047904549496831, ply_count=3, locs=(106, 110)),
  <Action.WSW: -11>),
 (Isolation(board=41198632737088974515941243613079551, ply_count=3, locs=(108, 83)),
  <Action.ESE: -15>),
 (Isolation(board=41198484184284260270308255718172671, ply_count=3, locs=(108, 83)),
  <Action.ESE: -15>),
 (Isolation(board=41523161203939122082683630076618751, ply_count=3, locs=(31, 17)),
  <Action.ESE: -15>),
 (Isolation(board=41523161203937941491027730440906719, ply_count=3, locs=(45, 5)),
  <Action.WNW: 15>),
 (Isolation(board=41523161203939122082683626855589879, ply_count=3, locs=(30, 32)),
  <Action.WSW: -11>),
 (Isolation(board=41523161203939122073672032775563263, ply_count=3, locs=(31, 53)),
  <Action.SSW: -25>),
 (Isolation(board=41520625901529739768237212624218111, ply_count=3, locs=(55, 101)),
  <Action.ENE: 11>),
 (Isolation(board=4152316120393912207818003

In [56]:
agent1 = isolation.Agent(custom.CustomPlayer, "CustomPlayer1")
agent2 = isolation.Agent(custom.CustomPlayer, "CustomPlayer2")

state = isolation.isolation.Isolation()
time_limit = 150
match_id = 0
winner, game_history, match_id = isolation.play(((agent1, agent2),
                                                state,
                                                time_limit,
                                                match_id))

In [59]:
winner.name

'CustomPlayer1'

In [61]:
game_history

[28,
 29,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.WSW: -11>,
 <Action.WSW: -11>,
 <Action.SSW: -25>,
 <Action.SSW: -25>,
 <Action.SSW: -25>,
 <Action.SSW: -25>,
 <Action.WSW: -11>,
 <Action.WSW: -11>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.WNW: 15>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.SSE: -27>,
 <Action.SSE: -27>,
 <Action.SSW: -25>,
 <Action.WNW: 15>,
 <Action.SSW: -25>,
 <Action.NNE: 25>,
 <Action.WNW: 15>,
 <Action.WSW: -11>,
 <Action.NNW: 27>,
 <Action.ESE: -15>,
 <Action.NNE: 25>,
 <Action.SSW: -25>,
 <Action.NNE: 25>,
 <Action.SSW: -25>,
 <Action.SSE: -27>,
 <Action.WNW: 15>,
 <Action.ENE: 11>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.NNW: 27>,
 <Action.WSW: -11>,
 <Action.ESE: -15>,
 <Action.WNW: 15>,
 <Action.SSW: -25>,
 <Action.WSW: -11>,
 <Action.WSW: -11>,
 <Action.SSW: -25>,
 <Action.ESE: -15>,
 <Action.ESE: -15>,
 <Action.ESE: -15>,
 <Action.ESE: -15>]

In [62]:
len(game_history)

52

In [ ]:
def process_game_history(state, game_history, book, winner_id):
    """ Given an initial state, and a list of actions, this function iterates
    through the resulting states of the actions and updates count of wins in 
    the state/action book"""
    

## IDEA: Iterative Time Limit increase